In [19]:
import pandas as pd
import paramiko
import os
import re
import json


In [ ]:
# load in all the csv files in data/kv21_resultater folder
path = "data/kv21_resultater/raw"

parti_info = json.load(open("data/partier.json"))
kommune_info = json.load(open("data/kommuner.json"))

super_df = pd.DataFrame()

csv_files = [os.path.join(path, file) for file in os.listdir(path) if file.endswith(".csv")]
for file in csv_files:
    df = pd.read_csv(file, sep=";")

    total_votes = df["Stemmetal"].sum()

    df = df.groupby(["Bogstavbetegnelse","Listenavn"]).agg({"Stemmetal":"sum"}).reset_index().sort_values(by="Stemmetal", ascending=False)
    df["Procent"] = df["Stemmetal"] / total_votes * 100

    def get_standard_party_name(row):
        key = str(row["Bogstavbetegnelse"]).strip().upper()
        for parti in parti_info:                  # ensure parti_info is a list of dicts
            if key == str(parti["listebogstav"]).strip().upper():
                # return standardized (name, bogstav)
                return parti["navn"], parti["bogstav"]
        # fallback: keep original values, but return a pair
        return row["Listenavn"], row["Bogstavbetegnelse"]

    df[["Listenavn", "bogstav"]] = df.apply(get_standard_party_name, axis=1, result_type="expand")  

    # get the kommune name from the file name by grabbing the part between the second and third underscore
    df['kommune_navn'] = file.split("_")[3]
    df['kommune_id'] = None
    df['kommune_dagi_id'] = None

    #rename and reorder the columns: 
    #kommune_id,kommune_dagi_id,kommune_navn,listebogstav,partier,bogstav,procent_25,stemmer_25
    df = df[["kommune_id","kommune_dagi_id","kommune_navn","Bogstavbetegnelse","Listenavn","bogstav","Procent","Stemmetal"]]
    df.columns = ["kommune_id","kommune_dagi_id","kommune_navn","listebogstav","partier","bogstav","procent_21","stemmer_21"]

    super_df = pd.concat([super_df, df], ignore_index=True)

In [79]:
# find the corresponding kommune_id and kommune_dagi_id from kommune_info
for index, row in super_df.iterrows():
    kommune_navn = row["kommune_navn"]
    for kommune in kommune_info:
        if kommune_navn.lower() == kommune["kommune_navn"].lower():
            super_df.at[index, "kommune_id"] = kommune["kommune_id"]
            super_df.at[index, "kommune_dagi_id"] = kommune["kommune_dagi_id"]
            break

In [84]:
super_df

,kommune_id,kommune_dagi_id,kommune_navn,listebogstav,partier,bogstav,procent_21,stemmer_21
0,751,389194,Aarhus,A,Socialdemokratiet,S,28.739157,55295
1,751,389194,Aarhus,C,Konservative,K,14.592808,28077
2,751,389194,Aarhus,F,SF,SF,13.741470,26439
3,751,389194,Aarhus,V,Venstre,V,11.868318,22835
4,751,389194,Aarhus,Ø,Enhedslisten,EL,9.897975,19044
...,...,...,...,...,...,...,...,...
1113,710,389188,Favrskov,B,Radikale,R,3.781591,1007
1114,710,389188,Favrskov,D,Nye Borgerlige,D,3.537497,942
1115,710,389188,Favrskov,O,Dansk Folkeparti,DF,3.511210,935
1116,710,389188,Favrskov,L,Samlingspartiet Favrskov,L,1.186676,316


In [ ]:
#super_df.to_csv("data/kv21_resultater/parti_resultater.csv", index=False)

In [2]:
# FTP login information
host = "data.valg.dk"
port = 22
username = "Valg"
password = "Valg"

# Forbindelse til FTP-server
transport = paramiko.Transport((host, port))
transport.connect(username=username, password=password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [9]:
sftp.listdir('data/regionsrådsvalg-134-18-11-2025/verifikation/valgresultater')

['valgresultater-Regionsrådsvalg-Københavns_Kommune-1__-190820250843.json.hash',
 'valgresultater-Regionsrådsvalg-Københavns_Kommune-1__-190820250843.json',
 'valgresultater-Regionsrådsvalg-Københavns_Kommune-2__Nord-190820250843.json.hash',
 'valgresultater-Regionsrådsvalg-Københavns_Kommune-2__Nord-190820250843.json',
 'valgresultater-Regionsrådsvalg-Københavns_Kommune-3__Nordvest-190820250843.json.hash',
 'valgresultater-Regionsrådsvalg-Københavns_Kommune-3__Nordvest-190820250843.json']

In [12]:
with sftp.open('data/regionsrådsvalg-134-18-11-2025/verifikation/valgresultater/valgresultater-Regionsrådsvalg-Københavns_Kommune-3__Nordvest-190820250843.json') as f:
    text = f.read().decode('latin-1')  # or 'windows-1252'
    data = json.loads(text)

In [13]:
data

{'Valgdag': '18-11-2025',
 'Valgart': 'RegionsrÃ¥dsvalg',
 'FrigivelsesTidspunktUTC': '19-08-2025 06:43:57',
 'Storkreds': 'KÃ¸benhavn',
 'Storkredsnummer': 1,
 'Opstillingskreds': 'Ã\x98sterbro',
 'OpstillingskredsDagiId': 1,
 'Kommune': 'KÃ¸benhavn',
 'Kommunekode': 101,
 'AfstemningsomrÃ¥deNummer': 6,
 'AfstemningsomrÃ¥deDagiId': 706166,
 'AfstemningsomrÃ¥de': '3. Nordvest',
 'UgyldigeFremmÃ¸dteStemmerÃ\x85rsager': [],
 'UgyldigeBrevstemmerÃ\x85rsager': [],
 'Resultatart': 'IngenResultater',
 'AfgivneStemmer': 0,
 'GyldigeStemmer': 0,
 'UgyldigeStemmerUdoverBlanke': 0,
 'BlankeUgyldigeFremmÃ¸dteStemmer': 0,
 'BlankeUgyldigeBrevstemmer': 0,
 'AfgivneStemmerDifferenceFraForrigeValg': None,
 'GyldigeStemmerDifferenceFraForrigeValg': None,
 'UgyldigeStemmerDifferenceFraForrigeValg': None,
 'BlankeStemmerDifferenceFraForrigeValg': None,
 'AntalStemmeberettigedeVÃ¦lgereDifferenceFraForrigeValg': None,
 'AntalStemmeberettigedeVÃ¦lgere': 40000}

In [ ]:
# with sftp.open('data/kommunalvalg-134-18-11-2025/geografi/Afstemningsomraade-291020251900.json') as f:
#     data = json.load(f)

FileNotFoundError: [Errno 2] No such file

In [69]:
afstemningssteder = []

for d in data:

    afstemningssteder.append({
        'dagi_id': d['Dagi_id'],
        'navn': d['Navn'],
        'nummer': d['Nummer'],
        'afstemningssted_navn': d['Afstemningssted']['Navn'],
        'kommune_id': d['Kommunekode'],
        'opstillingskreds_nummer': d['Opstillingskredsnummer'],
        'opstillingskreds_dagi_id': d['Opstillingskreds_Dagi_id'],
        'afstemningssted_adresse': d['Afstemningssted']['Adgangsadresse']['Adressebetegnelse']
    })

df_afstemningssteder = pd.DataFrame(afstemningssteder)
df_afstemningssteder      

,dagi_id,navn,nummer,afstemningssted_navn,kommune_id,opstillingskreds_nummer,opstillingskreds_dagi_id,afstemningssted_adresse
0,709812,"Langå, Svindinge Og Øksendrup",5,Svindinge Forsamlingshus,450,46,403614,"Strædet 2, Svindinge, 5853 Ørbæk"
1,748590,Nymarken,3,Nymarksskolen,440,46,403614,"Nymarken 47, Hundslev, 5300 Kerteminde"
2,706574,Kerteminde,2,Kerteminde Idrætscenter,440,46,403614,"Enggade 19, 5300 Kerteminde"
3,703760,Skellerup,8,Skellerup Forsamlingshus,450,46,403614,"Skellerup Byvej 16, 5540 Ullerslev"
4,703754,Nyborghallen,2,Nyborghallen,450,46,403614,"Halvej 1, 5800 Nyborg"
...,...,...,...,...,...,...,...,...
1309,707512,Sig,4,"Birgittegården, Rød Stue",573,54,403620,"Falkevej 4A, Sig, 6800 Varde"
1310,710690,Ølgod,30,Ølgod Hallen - Bemærk NYT afstemningssted,573,54,403620,"Skolegade 18, 6870 Ølgod"
1311,708972,Blåvand,19,Blåvand Borgerhus,573,54,403620,"Blåvandvej 30A, 6857 Blåvand"
1312,704694,Agerbæk,23,Agerbæk Hotel,573,54,403620,"Storegade 7, 6753 Agerbæk"


In [2]:
# load in the local kommune mapping file in json format
with open('data/kommuner.json') as f:
    kommune_info = json.load(f)

In [81]:
# join the kommune names to the afstemningssteder dataframe
df_afstemningssteder['kommune_navn'] = df_afstemningssteder['kommune_id'].astype(str).map(
    {k['kommune_id']: k['kommune_navn'] for k in kommune_info}
)

# map the kommune dagi ids
df_afstemningssteder['kommune_dagi_id'] = df_afstemningssteder['kommune_id'].astype(str).map(
    {k['kommune_id']: k['kommune_dagi_id'] for k in kommune_info}
)

# round the dagi id to integer and skip any errors
df_afstemningssteder['kommune_dagi_id'] = pd.to_numeric(df_afstemningssteder['kommune_dagi_id'], errors='coerce').round().astype('Int64')

In [82]:
df_afstemningssteder

,dagi_id,navn,nummer,afstemningssted_navn,kommune_id,opstillingskreds_nummer,opstillingskreds_dagi_id,afstemningssted_adresse,kommune_navn,kommune_dagi_id
0,709812,"Langå, Svindinge Og Øksendrup",5,Svindinge Forsamlingshus,450,46,403614,"Strædet 2, Svindinge, 5853 Ørbæk",Nyborg,389160
1,748590,Nymarken,3,Nymarksskolen,440,46,403614,"Nymarken 47, Hundslev, 5300 Kerteminde",Kerteminde,389159
2,706574,Kerteminde,2,Kerteminde Idrætscenter,440,46,403614,"Enggade 19, 5300 Kerteminde",Kerteminde,389159
3,703760,Skellerup,8,Skellerup Forsamlingshus,450,46,403614,"Skellerup Byvej 16, 5540 Ullerslev",Nyborg,389160
4,703754,Nyborghallen,2,Nyborghallen,450,46,403614,"Halvej 1, 5800 Nyborg",Nyborg,389160
...,...,...,...,...,...,...,...,...,...,...
1309,707512,Sig,4,"Birgittegården, Rød Stue",573,54,403620,"Falkevej 4A, Sig, 6800 Varde",Varde,389173
1310,710690,Ølgod,30,Ølgod Hallen - Bemærk NYT afstemningssted,573,54,403620,"Skolegade 18, 6870 Ølgod",Varde,389173
1311,708972,Blåvand,19,Blåvand Borgerhus,573,54,403620,"Blåvandvej 30A, 6857 Blåvand",Varde,389173
1312,704694,Agerbæk,23,Agerbæk Hotel,573,54,403620,"Storegade 7, 6753 Agerbæk",Varde,389173


In [83]:
# for each kommune dagi id generate a csv file with the afstemningssteder for that kommune
output_dir = 'data/struktureret/kv/valgresultater/afstemningssteder/'

os.makedirs(output_dir, exist_ok=True)

num = 0
for kommune_id, group in df_afstemningssteder.groupby('kommune_id'):
    kommune_navn = group['kommune_navn'].iloc[0].lower().replace(' ', '_')
    output_file = os.path.join(output_dir, f'{kommune_id}_{kommune_navn}_afstemningsområde.csv')
    group['største_parti'] = ''  # placeholder column for største parti
    group.to_csv(output_file, index=False)
    num += 1
print(f'Wrote afstemningssteder for {num} kommuner to {output_dir}')

Wrote afstemningssteder for 98 kommuner to data/struktureret/kv/valgresultater/afstemningssteder/


In [84]:
# also generate a file for each kommune in the folder data/struktureret/kv/valgresultater/kommune/
output_dir_kommune = 'data/struktureret/kv/valgresultater/kommune/'

os.makedirs(output_dir_kommune, exist_ok=True)
# the file should have the columns: kommune_id, kommune_dagi_id, kommune_navn, parti, procent_25, stemmer_25, procent_21, stemmer_21

for kommune_id, group in df_afstemningssteder.groupby('kommune_id'):
    kommune_navn = group['kommune_navn'].iloc[0].lower().replace(' ', '_')
    kommune_dagi_id = group['kommune_dagi_id'].iloc[0]
    output_file = os.path.join(output_dir_kommune, f'{kommune_id}_{kommune_navn}_kommune.csv')
    df_kommune = pd.DataFrame([{
        'kommune_id': kommune_id,
        'kommune_dagi_id': kommune_dagi_id,
        'kommune_navn': group['kommune_navn'].iloc[0],
        'partier': '',
        'procent_25': '',
        'stemmer_25': '',
        'procent_21': '',
        'stemmer_21': ''
    }])
    df_kommune.to_csv(output_file, index=False)

In [16]:
regioner_geo = json.load(open("data/shapes/regioner_afstemningsområder.geojson"))

In [18]:
# the file has shapes for five regions and their afstemningsområder
old_regioner = ['Region Hovedstaden', 'Region Sjælland', 'Region Syddanmark', 'Region Midtjylland', 'Region Nordjylland']
new_regioner = ['Region Østdanmark', 'Region Syddanmark', 'Region Midtjylland', 'Region Nordjylland']

# split the geojson into separate files for each new region
for region in new_regioner:
    region_geo = {
        "type": "FeatureCollection",
        "features": []
    }
    for feature in regioner_geo['features']:
        if region == 'Region Østdanmark':
            if feature['properties']['regionsnavn'] in ['Region Hovedstaden', 'Region Sjælland']:
                region_geo['features'].append(feature)
        else:
            if feature['properties']['regionsnavn'] == region:
                region_geo['features'].append(feature)
    with open(f"data/shapes/{region.lower().replace(' ', '_')}_afstemningsområder.geojson", "w") as f:
        json.dump(region_geo, f)

In [3]:
# turn data/kommuner.json into a dataframe
df_kommuner = pd.DataFrame(kommune_info)

In [6]:
df_kommuner.to_clipboard()

In [ ]:
# loop over each kommune in kommune info and print the kommune_id and kommune_navn
for kommune in kommune_info:
    # create a dataframe with the columns    kommune_id kommune_navn  andel_af_afstemningssteder_talt  borgmester
    df = pd.DataFrame({
        'kommune_id': [kommune['kommune_id']],
        'kommune_navn': [kommune['kommune_navn']],
        'andel_af_afstemningssteder_talt': 0,
        'borgmester': "Ikke afgjort"
    })
    # save the dataframe to a csv file in data/struktureret/kv/valgresultater/status/
    df.to_csv(f"data/struktureret/kv/valgresultater/status/{kommune['kommune_id']}_{kommune['kommune_navn'].lower()}_status.csv", index=False)